In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D,Dropout
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

Using TensorFlow backend.


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [6]:
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: cat_dog_sub.zip, id: 1h8fuz_ayrQUVqc3jEIU6SliGSm7UbHHb
title: Colab Notebooks, id: 1ELYe8t8R6V98MOn_8zqVkGLFL603hFvB
title: August Baking, id: 1XPVr5Tt2KPNmfze2qWgjkgjBIEd92bDuEjGg25QLZz4
title: mnist_data.zip, id: 1qSWcT72_Yd2wvv_aMEvvYZWFfIGt9XSM
title: August baking, id: 1yELvZ-x0aHME7J-1Jw6y1aPvTUwzPYTywtaTWBfK0RY
title: August baking.xlsx, id: 1G3USkxinhVNfG0XPFostxnrENNohfmLz
title: Instructor Survey (Responses), id: 1c3WNh6xSD5bzgFuX3Qmx7MAVndCBd5EbK3esEtDnXts
title: Instructor Survey, id: 1MYkkR1x125ZjxJYZabyTaX0qJlfT4CJK6lSi6KBiISI
title: Affine Exercise.R, id: 0B8drO-I4mTyudUsySXFrUFVYZlhScXRKOVJVR1JFRTJPTnFV
title: draft curriculum, id: 1TRTme8a9qqz2T5uDkxsyDDV1J5brtIH-3UbJ4GAuwlE
title: Google Buzz, id: 1W7W4-9YrhV3bZMWs--Ri0UYzpKFN6fEQICybRwRc


In [0]:
downloaded = drive.CreateFile({'id': '1h8fuz_ayrQUVqc3jEIU6SliGSm7UbHHb'})
downloaded.GetContentFile('cat_dog.zip')

In [8]:
!unzip cat_dog.zip

Archive:  cat_dog.zip
   creating: cat_dog_sub/
  inflating: cat_dog_sub/.DS_Store   
   creating: __MACOSX/
   creating: __MACOSX/cat_dog_sub/
  inflating: __MACOSX/cat_dog_sub/._.DS_Store  
   creating: cat_dog_sub/test/
   creating: cat_dog_sub/test/cat/
  inflating: cat_dog_sub/test/cat/5109.jpg  
   creating: __MACOSX/cat_dog_sub/test/
   creating: __MACOSX/cat_dog_sub/test/cat/
  inflating: __MACOSX/cat_dog_sub/test/cat/._5109.jpg  
  inflating: cat_dog_sub/test/cat/1421.jpg  
  inflating: __MACOSX/cat_dog_sub/test/cat/._1421.jpg  
  inflating: cat_dog_sub/test/cat/604.jpg  
  inflating: __MACOSX/cat_dog_sub/test/cat/._604.jpg  
  inflating: cat_dog_sub/test/cat/7087.jpg  
  inflating: __MACOSX/cat_dog_sub/test/cat/._7087.jpg  
  inflating: cat_dog_sub/test/cat/8149.jpg  
  inflating: __MACOSX/cat_dog_sub/test/cat/._8149.jpg  
  inflating: cat_dog_sub/test/cat/5479.jpg  
  inflating: __MACOSX/cat_dog_sub/test/cat/._5479.jpg  
  inflating: cat_dog_sub/test/cat/1151.jpg  
  inflati

In [0]:
import os

In [10]:
os.listdir()

['.config',
 'cat_dog.zip',
 'mnist_data',
 '__MACOSX',
 'adc.json',
 'mnist.zip',
 'cat_dog_sub',
 'sample_data']

In [11]:
base_model=MobileNet(weights='imagenet',include_top=False)

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:207: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


17227776/17225924 [==============================] - 1s 0us/step


In [12]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128       
__________

In [0]:
CLASSES = 2
x = base_model.output
x = GlobalAveragePooling2D(name='avg_pool')(x)
x = Dropout(0.4)(x)
predictions = Dense(CLASSES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [0]:
for layer in base_model.layers:
    layer.trainable = False
      
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128       
__________

In [17]:
os.listdir('cat_dog_sub')

['train', '.DS_Store', 'test']

In [18]:
WIDTH = 299
HEIGHT = 299
BATCH_SIZE = 32
TRAIN_DIR=r'cat_dog_sub/train'
TEST_DIR=r'cat_dog_sub/test'
# data prep
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(HEIGHT, WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='categorical')
    
validation_generator = validation_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(HEIGHT, WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

Found 1999 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [19]:
EPOCHS = 5
BATCH_SIZE = 32
STEPS_PER_EPOCH = 320
VALIDATION_STEPS=64
model.fit_generator(
    train_generator,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=validation_generator,
    validation_steps=VALIDATION_STEPS)

Epoch 1/5
320/320 [==============================] - 295s 923ms/step - loss: 0.2746 - acc: 0.8821 - val_loss: 0.3269 - val_acc: 0.8627
Epoch 2/5
320/320 [==============================] - 287s 896ms/step - loss: 0.1601 - acc: 0.9364 - val_loss: 0.2044 - val_acc: 0.9144
Epoch 3/5
320/320 [==============================] - 286s 895ms/step - loss: 0.1520 - acc: 0.9396 - val_loss: 0.0882 - val_acc: 0.9700
Epoch 4/5
320/320 [==============================] - 286s 895ms/step - loss: 0.1574 - acc: 0.9399 - val_loss: 0.1050 - val_acc: 0.9557
Epoch 5/5
320/320 [==============================] - 286s 894ms/step - loss: 0.1544 - acc: 0.9377 - val_loss: 0.0905 - val_acc: 0.9665


In [0]:
from tensorflow.python.keras.preprocessing import image

In [0]:
img = image.load_img(r'cat_dog_sub/test/dog/182.jpg', target_size=(299,299))

In [0]:
x = image.img_to_array(img)
x = preprocess_input(x)
x = np.expand_dims(x, axis=0)
pred = model.predict(x)[0]

In [26]:
pred

array([1.2811643e-05, 9.9998724e-01], dtype=float32)